In [1]:
# Built-in library
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import Any, Optional, Union

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as pltife

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=2)
from settings import refresh_settings  # noqa: E402

settings = refresh_settings()

/Users/neidu/Desktop/Projects/Personal/My_Projects/AI-Tutorials


In [4]:
from typing import TypedDict

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI, OpenAI
from langgraph.graph import END, START, StateGraph

## 1.) State

- This is the central concept in Langgraph. It represents all the information that flows thru the application.

In [ ]:
### Define State
class EmailState(TypedDict):
    email: dict[str, Any]  # contains subject, sender, body, etc.

    # analysis and decision
    is_spam: bool | None

    # reason for the marking as spam
    spam_reason: str | None

    # category of the email (inquiry, complaint, etc.)
    email_category: str | None

    # response generation
    email_draft: str | None

    # processing metadata
    message: list[dict[str, Any]]  # for tracking conversation with LLM

In [6]:
import asyncio
from enum import Enum

import instructor
import nest_asyncio
import ollama
from openai import AsyncOpenAI, OpenAI
from pydantic import BaseModel, Field


class ModelEnum(str, Enum):
    DEEPSEEK_R1_1p5B_local = "deepseek-r1:1.5b"
    GEMMA_3p0_1B_local = "gemma3:1b"
    GWEN_2p5_3B_local = "qwen2.5:3b"
    GWEN_2p5_7B_local = "qwen2.5:7b-instruct-q3_K_M"
    GWEN_3p0_4B_local = "qwen3:4b-q4_K_M"
    GWEN_3p0_7B_local = "qwen3:8b-q4_K_M"
    LLAMA_3p1_8B = "llama-3.1-8b-instant"
    LLAMA_4_SCOUT_17B = "meta-llama/llama-4-scout-17b-16e-instruct"
    GWEN_32B = "qwen-qwq-32b"
    BASE_MODEL = "qwen3:4b-q4_K_M"


client: AsyncOpenAI = AsyncOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",  # required, but unused
)
model: str = ModelEnum.GWEN_3p0_7B_local.value

# Patch the client
# client = instructor.from_openai(
#     ollama_client,
#     mode=instructor.Mode.JSON,
# )


# client = AsyncOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     api_key=settings.GROQ_API_KEY.get_secret_value(),
# )

# model: str = ModelEnum.LLAMA_17B.value
model

'qwen3:8b-q4_K_M'

In [7]:
nest_asyncio.apply()


async def chat_completion(
    message: dict[str, Any],
    client: AsyncOpenAI = client,
    model: str = model,
    print_chunk: bool = False,
) -> str:
    """This function takes a message and returns a response from the LLM."""
    messages: list[dict[str, Any]] = [
        {
            "role": "assistant",
            "content": "</no_think> You're a AI assistant, Alfred the butler!",
        }
    ]
    messages.append(message)

    result = await client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=4_096,
        temperature=0,
        stream=True,
        seed=2,
    )

    full_response: str = ""

    async for chunk in result:
        # Check if the chunk contains content and append it
        if chunk.choices[0].delta and chunk.choices[0].delta.content:
            if print_chunk:
                print(chunk.choices[0].delta.content, end="", flush=True)
            full_response += chunk.choices[0].delta.content
    return full_response


message: dict[str, Any] = {
    "role": "user",
    "content": "Tell me a short funny joke about Game of Thrones TV series.",
}

result = asyncio.run(chat_completion(message=message, print_chunk=True))

# print(result)

<think>

</think>

Ah, a joke about *Game of Thrones*... how delightful! Here's one:

I told my wife I was going to the Citadel to study magic.  
She said, "You mean like the *Game of Thrones* magic?"  
I replied, "No, I mean the real magic—like getting a better Wi-Fi signal!"  
She said, "You're a *dragon* of a husband!"  

*Pours a glass of wine and smiles mysteriously.*

### 2.) Nodes

- These are Python functions that represent the main logic of the application. 
- Each node: 
  - takes the state as input.
  - performs some operation.
  - returns pdates to the state.

In [8]:
def extract_response(result: str) -> dict[str, Any]:
    """Extract the response from the result."""
    try:
        return json.loads(result.replace("<think>", "").replace("</think>", "").strip())
    except json.JSONDecodeError:
        return {}


async def read_email(state: EmailState) -> dict[str, Any]:
    """Alfred reads and logs the incoming email."""
    email: dict[str, Any] = state["email"]

    # Some preprocessing steps
    print(
        f"Alfred is processing an email from {email['sender']!r} with "
        f"subject {email['subject']!r}.\n\n"
    )

    # No state changes neede here
    return {}


async def classify_email(state: EmailState) -> dict[str, Any]:
    """Alfred uses an LLM to classify the email as spam or not."""
    email: dict[str, Any] = state["email"]
    categories: list[str] = [
        "inquiry",
        "complaint",
        "thank you",
        "request",
        "information",
        "other",
    ]

    # Prepare the prompt
    prompt: str = f"""
    <prompt>
        As Alfred the butler, analyze this email and determine if it's 
        <type> **spam** or **ham** </type>.

        <email>
            From: {email["sender"]}
            Subject: {email["subject"]}
            Body: {email["body"]}
        </email>

        <instruction>
            1. Determine if this email is <type> **spam** or **ham** </type>.
            2. If it's spam, explain why.
            3. If it's ham, categorize it as <categories> {categories} </categories>.
        </instruction>

        # Example Output:
        <response>
            {{
                "type": "ham",
                "reason": "null",
                "email_category": "inquiry",
            }}
        </response>

        <instruction>
            Return the response in JSON format shown above. Remember to include your reason
            if it's a spam email.
        </instruction>

    </prompt>
    """

    # Call the LLM
    response: str = await chat_completion(message={"role": "user", "content": prompt})
    # print(f"Response: {response}")  # for debugging

    # Parse response
    response_text: dict[str, Any] = extract_response(response)
    is_spam: bool = response_text["type"] == "spam"
    # print(f"Response: {response_text}")  # for debugging

    # Extract reason if spam
    spam_reason: str | None = None
    try:
        if is_spam:
            spam_reason = response_text["reason"]
    except Exception as e:
        print(f"Error: {e}\n{type(e)}")

    # Determine the category
    email_category: str | None = None
    if not is_spam:
        email_category = response_text["email_category"]

    # Update messages for tracking
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response},
    ]

    # Return state updates
    return {
        "is_spam": is_spam,
        "spam_reason": spam_reason,
        "email_category": email_category,
        "messages": new_messages,
    }


async def handle_spam(state: EmailState) -> dict[str, Any]:
    """Alfred discards spam emails."""
    print(
        f"Alfred has marked the email as spam. \nReason: {state['spam_reason']}. \n"
        "The email has been moved to the spam folder."
    )

    # No state changes needed here
    return {}


async def draft_response(state: EmailState) -> dict[str, Any]:
    """Alfred drafts a response to the email."""
    # print(f"{state = }")
    email: dict[str, Any] = state["email"]

    # Prepare the prompt
    prompt: str = f"""
    <prompt>
        As Alfred the butler, draft a response to this email.

        <email>
            From: {email["sender"]}
            Subject: {email["subject"]}
            Body: {email["body"]}
        </email>

        This email has been categorized as {state["email_category"]}.

        <instruction>
            Draft a brief response that Mr. Neidu can review and personalize before sending.
        </instruction>

    </prompt>
    """

    # Call the LLM
    response: str = await chat_completion(message={"role": "user", "content": prompt})

    # Update messages for tracking
    new_message: dict[str, Any] = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response},
    ]

    # Return state updates
    return {
        "email_draft": response,
        "messages": new_message,
    }


async def notify_user(state: EmailState) -> dict[str, Any]:
    """Alfred sends a notification to the user."""
    # print(f"{state = }")  # for debugging

    email: dict[str, Any] = state["email"]

    print("\n" + "=" * 50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"Category: {state['email_category']}")
    print("\nI've drafted a response for your review.")
    print("-" * 50)
    print(state["email_draft"])
    print("=" * 50 + "\n")

    return {}

### 3.) Define Routing Logic

In [9]:
def route_email(stete: EmailState) -> str:
    """Determine the next step to take."""
    if stete["is_spam"]:
        return "spam"

    return "ham"

### 4.) Create StateGraph and Define Edges

In [10]:
# Create the graph
email_graph = StateGraph(EmailState)

# Add nodes
email_graph.add_node("read_email", read_email)
email_graph.add_node("classify_email", classify_email)
email_graph.add_node("handle_spam", handle_spam)
email_graph.add_node("draft_response", draft_response)
email_graph.add_node("notify_user", notify_user)

# Start the edges
email_graph.add_edge(START, "read_email")
email_graph.add_edge("read_email", "classify_email")

# Add conditional branches
email_graph.add_conditional_edges(
    "classify_email", route_email, {"spam": "handle_spam", "ham": "draft_response"}
)

# Add the fnal edges
email_graph.add_edge("handle_spam", END)
email_graph.add_edge("draft_response", "notify_user")
email_graph.add_edge("notify_user", END)

# Compile the graph
compiled_graph = email_graph.compile()

### Run The Application

In [ ]:
# Example ham email
ham_email: dict[str, Any] = {
    "sender": "john.smith@example.com",
    "subject": "Meeting Request",
    "body": "Dear Mr. Neidu, I hope this email meets you well. We have a demo next week, "
    "and would like to inform you about the services we offer. Can we schedule a "
    "meeting to discuss the details? Thank you for your time."
    "Best regards, John Smith",
}

ham_email_2: dict[str, Any] = {
    "sender": "john.smith@example.com",
    "subject": "Following Up: Quick Idea on Improving Workflow Efficiency",
    "body": "Dear Mr. Neidu,My name is John Smith, and I'm with Innovate Solutions Inc. "
    "I'm reaching out because I believe our solution could offer significant value in "
    "streamlining your operational workflows. We are conducting brief demos next week, "
    "and I would welcome the opportunity to show you how our workflow automation platform works "
    "and discuss how it might specifically benefit your operations team."
    "Would you be available for a quick, no-obligation introductory call sometime "
    "next week to explore this further? Please let me know what time works best for you."
    "Thank you for your time and consideration. \n"
    "Yours sincerely, John Smith",
}

# Example spam email
spam_email: dict[str, Any] = {
    "sender": "winner@lottery-intl.com",
    "subject": "YOU HAVE WON $5,000,000!!!",
    "body": "CONGRATULATIONS! You have been selected as the winner of our international "
    "lottery! To claim your $5,000,000 prize, please send us your bank details and a "
    "processing fee of $100.",
}

# Process the ham email
print("\nProcessing ham email...")
try:
    ham_result = await compiled_graph.ainvoke(
        {
            "email": ham_email,
            "is_spam": None,
            "spam_reason": None,
            "email_category": None,
            "email_draft": None,
            "messages": [],
        }
    )
except Exception as e:
    print(f"Error: {e}\n{type(e)}")


Processing ham email...
Alfred is processing an email from 'john.smith@example.com' with subject 'Meeting Request'.



Sir, you've received an email from john.smith@example.com.
Subject: Meeting Request
Category: inquiry

I've drafted a response for your review.
--------------------------------------------------
<think>

</think>

**Subject:** Re: Meeting Request  

Dear Mr. Smith,  

Thank you for your email and for reaching out to discuss the upcoming demo. I am pleased to hear about your services and would be happy to arrange a convenient time for a meeting. Please let me know your preferred date and time, and I will ensure everything is prepared accordingly.  

Kind regards,  
**Alfred**  
*Personal Assistant to Mr. Neidu*



In [12]:
console.log(ham_result)

[12:49:12] {                                                                                         ]8;id=711730;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_51663/494640920.py\494640920.py]8;;\:]8;id=212631;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_51663/494640920.py#1\1]8;;\
               'email': {                                                                                          
                   'sender': 'john.smith@example.com',                                                             
                   'subject': 'Meeting Request',                                                                   
                   'body': 'Dear Mr. Neidu, I hope this email meets you well. We have a demo next                  
           week, and would like to inform you about the services we offer. Can we schedule a meeting               
           to discuss the details? Thank you for your time.Best regards, John Smith'                               
               },                                                                                                  
               'email_category': 'inquiry',                                                                        
               'spam_reason': None,                                                                                
               'is_spam': False,                                                                                   
               'email_draft': '<think>\n\n</think>\n\n**Subject:** Re: Meeting Request  \n\nDear Mr.               
           Smith,  \n\nThank you for your email and for reaching out to discuss the upcoming demo. I               
           am pleased to hear about your services and would be happy to arrange a convenient time                  
           for a meeting. Please let me know your preferred date and time, and I will ensure                       
           everything is prepared accordingly.  \n\nKind regards,  \n**Alfred**  \n*Personal                       
           Assistant to Mr. Neidu*'                                                                                
           }                                                                                                       

In [13]:
# Process the spam email
print("\nProcessing spam email...")
try:
    spam_result = await compiled_graph.ainvoke(
        {
            "email": spam_email,
            "is_spam": None,
            "spam_reason": None,
            "email_category": None,
            "email_draft": None,
            "messages": [],
        }
    )
except Exception as e:
    print(f"Error: {e}")


Processing spam email...
Alfred is processing an email from 'winner@lottery-intl.com' with subject 'YOU HAVE WON $5,000,000!!!'.


Alfred has marked the email as spam. 
Reason: This email is a classic example of phishing and lottery scam. It uses urgent and enticing language to trick the recipient into providing sensitive information such as bank details. The sender is likely not a legitimate entity, and the request for a processing fee is a red flag for fraud.. 
The email has been moved to the spam folder.


In [15]:
console.log(spam_result, style="warning")

[12:52:00] {                                                                                        ]8;id=25164;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_51663/4245628018.py\4245628018.py]8;;\:]8;id=959893;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_51663/4245628018.py#1\1]8;;\
               'email': {                                                                                          
                   'sender': 'winner@lottery-intl.com',                                                            
                   'subject': 'YOU HAVE WON $5,000,000!!!',                                                        
                   'body': 'CONGRATULATIONS! You have been selected as the winner of our                           
           international lottery! To claim your $5,000,000 prize, please send us your bank details                 
           and a processing fee of $100.'                                                                          
               },                                                                                                  
               'email_category': None,                                                                             
               'spam_reason': 'This email is a classic example of phishing and lottery scam. It                    
           uses urgent and enticing language to trick the recipient into providing sensitive                       
           information such as bank details. The sender is likely not a legitimate entity, and the                 
           request for a processing fee is a red flag for fraud.',                                                 
               'is_spam': True,                                                                                    
               'email_draft': None                                                                                 
           }                                                                                                       

### Others



- Edges: These are connections between nodes. They represent the possible paths thru the graph.

- StateGraph: This is a container that holds the entire agent workflow.